## LangChain's Components

### Models and Prompts

In [ ]:
from langchain.llms import OpenAI
import os

os.environ["OPENAP_API_KEY"]

llm = openAI()
print(llm('tell me a joke'))

In [ ]:
from langchain import PromptTemplate

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "the cat is on the table", language="spanish"))

### Data Connections

#### Document loaders

In [ ]:
import csv

# Sample data
data = [
    ['Name', 'Age', 'City'],
    ['John', 25, 'New York'],
    ['Emily', 28, 'Los Angeles'],
    ['Michael', 22, 'Chicago']
]

# File name
file_name = 'sample.csv'

# Write data to csv file
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'Sample CSV file "{file_name}" generated and saved.')

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

#### Document splitters

In [ ]:
# Sample sentences about mountains and nature
content = """Amidst the serene landscape, towering mountains stand as majestic guardians of nature's beauty.
The cris mountain air carries whispers of tranquility, while the rustling leaves compose a symphony of wilderness.
Nature's palette paints the mountains with hues of green and brown, creating an awe-inspiring sight to behold.
As the sub ruses, it casts a golden glow on the mountain peaks, illuminating a world untouched and wild."""

# File name
file_name = 'mountain.txt'

# Write content to text file
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

#print(f'sample text file "{file_name}" generated and saved')

with open('mountain.txt') as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])
print(texts[0])
print(texts[1])
print(texts[2])

#### Text embedding models

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

embeddings_model = OpenAIEmbeddings(model = 'text-embedding-ada-002')

embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I have your name?",
        "Sure, Mario Rossi."
    ]
)

print("Embed documents:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

print("Embed query:")
print(f'Dimensions of the vector: {len(embedded_query)}')
print(f'Sample of the first 5 elements of the vector: {embedded_query[:5]}')

In [ ]:
# saving the conversation in a text file
# List of dialogue lines
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I have your name?",
    "Sure, Mario Rossi."
]

# File name
file_name = 'dialogue.txt'

# Write dialogue lines to text file
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'Dialogue text file "{file_name}" generated and saved.')


#### Vector stores

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

# Load the document, split it into chunks, embed each chunk and load it into the vector store.

raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator="\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
query = "what is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
print(documents[2])

#### Retrievers

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = db.as_retriever()


In [ ]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What was the reason for the call?"
qa.run(query)

### Memory

In [ ]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input":"hi, I'm looking for some ideas to write an essay in AI"}, {"output": "hello, what about writing on LLMs?"})

memory.load_memory_variables({})

In [ ]:
ConversationSummaryMemory.save_context?

### Chains

#### Simpe Chain

In [ ]:
from langchain import PromptTemplate, OpenAI, LLMChain

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict(sentence="the cat is on the table", language="spanish")